In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import scipy.fft as fft
import numpy as np

In [ ]:
def hat(t, center, width):
    return np.maximum(0, 1 - np.abs(t - center) / width)

def box(t, start, end):
    return np.where((t >= start) & (t <= end), 1, 0)

def gaussian(t, center, sigma, start, end):
    g = np.exp(-0.5 * ((t - center) / sigma) ** 2)
    g = g / (sigma * np.sqrt(2 * np.pi))
    return np.where((t >= start) & (t <= end), g, 0)

def f(t):
    #return np.sin(2 * np.pi * 1 * t) + 0.5 * np.sin(2 * np.pi * 5 * t)
    # hat from t=2 to t=4
    return np.where((t >= 0) & (t <= 4), hat(t, 2, 1), 0) + np.where((t > 4) & (t <= 8), np.exp(-1.5*(t-4)), 0) \
                + np.where((t > 8) & (t <= 10), t*0.02*np.sin(2 * np.pi * 3 * t) + 0.4, 0) \
                + 0.4*box(t, 11, 13) \
                + hat(t, 14, 0.2) + hat(t, 14.5, 0.1) + hat(t, 15, 0.05)

In [ ]:
nsamples = 1001
t_max = 16
t = np.linspace(0, t_max, nsamples, endpoint=False)
frequencies = fft.fftshift(fft.fftfreq(nsamples, d=(t[1] - t[0])))
dt = t[1] - t[0]

y = f(t)
fig, ax_time = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
ax_time.plot(t, y, label='Original Signal', color='blue', linewidth=1)
ax_time.set_title('Signal Sampling and Aliasing')
ax_time.set_xlabel('Time [s]')
ax_time.set_ylabel('Amplitude')
ax_time.grid(True)
ax_time.legend()
# set plot limits
ax_time.set_xlim(0, t_max)
ax_time.set_ylim(-0.1, 1.2)
plt.savefig('figs/samp_and_rec/function.png', dpi=300)
plt.show()

Y = fft.fftshift(fft.fft(y))  # scale by dt for CTFT units
fig, ax_freq = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
ax_freq.plot(frequencies, np.log(1 +np.abs(Y)), color='red', linewidth=1)
ax_freq.set_title('Frequency Spectrum of Original Signal')
ax_freq.set_xlabel('Frequency [Hz]')
ax_freq.set_ylabel('Magnitude')
ax_freq.grid(True)
# set plot limits
ax_freq.set_xlim(min(frequencies) - 1, max(frequencies) + 1)
ax_freq.set_ylim(0, 6)
plt.savefig('figs/samp_and_rec/function_spectrum.png', dpi=300)
plt.show()


# original samples
k_nsamples = 1024
for kname in ["box", "hat", "gaussian", "gaussian2"]:
    k_values = []
    t_kbox = []
    if kname == "box":
        t_kbox = np.linspace(-1.2, 1.2, k_nsamples, endpoint=False)
        k_values = box(t_kbox, -0.5, 0.5)
    elif kname == "hat":
        t_kbox = np.linspace(-2.2, 2.2, k_nsamples, endpoint=False)
        k_values = hat(t_kbox, 0, 1)
    elif kname == "gaussian":
        t_kbox = np.linspace(-2.2, 2.2, k_nsamples, endpoint=False)
        k_values = gaussian(t_kbox, 0, 0.5, -1.5, 1.5)
    elif kname == "gaussian2":
        t_kbox = np.linspace(-4.2, 4.2, k_nsamples, endpoint=False)
        k_values = gaussian(t_kbox, 0, 1.5, -5, 5)
    # zero-pad
    k_dt = t_kbox[1] - t_kbox[0]
    nfft = 32 * k_nsamples  # e.g., 16x padding
    k_ft = fft.fftshift(fft.fft(k_values, n=nfft)) * k_dt  # scale by dt for CTFT units
    frequencies_k = fft.fftshift(fft.fftfreq(nfft, d=k_dt))

    # limit frequency range
    mask = (frequencies_k >= -10) & (frequencies_k <= 10)
    frequencies_k = frequencies_k[mask]
    k_ft = k_ft[mask]

    #plot time-domain box kernel
    fig, ax_time = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
    ax_time.plot(t_kbox, k_values, color='green', linewidth=1)
    ax_time.set_title(f'{kname} Kernel in Time Domain')
    ax_time.set_xlabel('Time [s]')
    ax_time.set_ylabel('Amplitude')
    ax_time.grid(True)
    plt.savefig(f'figs/samp_and_rec/{kname}_kernel_time.png', dpi=300)
    plt.show()

    # plot frequency response
    fig, ax_freq = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
    # ax_freq.plot(frequencies_k, BOX_K, color='brown', linewidth=1)
    ax_freq.plot(frequencies_k, np.abs(k_ft), color='orange', linewidth=1)
    ax_freq.set_title(f'Frequency Power Spectrum of {kname} Kernel')
    ax_freq.set_xlabel('Frequency [Hz]')
    ax_freq.set_ylabel('Magnitude')
    ax_freq.grid(True)
    plt.savefig(f'figs/samp_and_rec/{kname}_kernel_spectrum.png', dpi=300)
    plt.show()

# plot box from -16 Hz to 16 Hz in freq domain [-32 Hz to 32 Hz]
box_f = np.linspace(-32, 32, 2048, endpoint=False)
BOX_K = box(box_f, -16, 16)
fig, ax_freq = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
ax_freq.plot(box_f, BOX_K, color='brown', linewidth=1)
ax_freq.set_title('Box Kernel in Frequency Domain')
ax_freq.set_xlabel('Frequency [Hz]')
ax_freq.set_ylabel('Amplitude')
ax_freq.grid(True)
plt.savefig('figs/samp_and_rec/box_kernel_on_freq_domain.png', dpi=300)
plt.show() 

# plot sinc from -32s to 32s in time domain
sinc_t = np.linspace(-32, 32, 2048, endpoint=False)
SINC_K = np.sinc(sinc_t / np.pi)
fig, ax_time = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
ax_time.plot(sinc_t, SINC_K, color='purple', linewidth=1)
ax_time.set_title('Sinc Kernel in Time Domain')
ax_time.set_xlabel('Time [s]')
ax_time.set_ylabel('Amplitude')
ax_time.grid(True)
plt.savefig('figs/samp_and_rec/sinc_kernel_on_time_domain.png', dpi=300)
plt.show()


In [ ]:
plt.close('all')
for resampled_rate in [2]:#[50, 20, 10, 2, 1]:
    # sequece of impulses
    i_t = np.where((np.array(range(0, nsamples))+1) % resampled_rate == 0, 1, 0)
    fig, ax = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
    # ax.plot(t, i_t, label='Impulse Train', color='blue')
    # filter for positive values
    t_it = t[i_t == 1]
    i_t_pos = i_t[i_t == 1]
    # add dashed  vertical line at sample points from 0 to 1
    for x in t_it:
        ax.plot([x, x], [0, 1], color='lightgray', linestyle='--', linewidth=0.8)
    ax.scatter(t_it, i_t_pos, label='Impulse Train', color='blue', s=10)
    ax.set_title('Impulse Train Sampling')
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Amplitude')
    ax.legend()
    # set plot limits
    ax.set_xlim(0, t_max)
    ax.set_ylim(-0.1, 1.2)
    plt.savefig(f'figs/samp_and_rec/impulse_train_{resampled_rate}.png', dpi=300)
    plt.show()

    # find impulse FT
    I_f = fft.fftshift(fft.fft(i_t))
    fig, ax = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
    ax.plot(frequencies, np.abs(I_f), label='Impulse Train Spectrum', color='red')
    ax.set_title('Frequency Spectrum of Impulse Train')
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('Magnitude')
    ax.legend()
    # set plot limits
    ax.set_xlim(min(frequencies) - 1, max(frequencies) + 1)
    plt.savefig(f'figs/samp_and_rec/impulse_spectrum_{resampled_rate}.png', dpi=300)
    plt.show()


    # get y samples at impulse locations
    y_samples = y * i_t
    # keep only the sampled values (non-zero)
    y_samples_pos = y_samples[y_samples != 0]
    t_samples_pos = t[y_samples != 0]
    # plot sampled signal
    fig, ax = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
    ax.plot(t, y, label='Original Signal', color='lightgray', linewidth=1)
    # if resampled_rate >= 10:
    #     ax.plot(t, y_samples, label='Sampled Signal', color='blue')
    ax.scatter(t_samples_pos, y_samples_pos, color='blue', s=10)
    ax.set_title('Sampled Signal')
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Amplitude')
    # set plot limits
    ax.set_xlim(0, t_max)
    ax.set_ylim(-0.1, 1.2)
    ax.legend()
    plt.savefig(f'figs/samp_and_rec/sampled_signal_{resampled_rate}.png', dpi=300)
    plt.show()


    Y_samples = fft.fftshift(fft.fft(y_samples))
    fig, ax = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
    ax.plot(frequencies, np.log(1 + np.abs(Y_samples)), color='red', linewidth=1)
    # ax.plot(frequencies,  np.abs(Y_samples), color='green', linewidth=1)
    ax.set_title('Frequency Spectrum of Sampled Signal')
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('Magnitude')
    # set plot limits
    ax.set_xlim(min(frequencies) - 1, max(frequencies) + 1)
    # ax.set_ylim(0, 6)
    ax.legend()
    plt.savefig(f'figs/samp_and_rec/sampled_signal_spectrum_{resampled_rate}.png', dpi=300)
    plt.show()

    # plot np.log(1 + np.abs(Y)) shifted by multiples of sampling frequency
    sampling_frequency = 1 / (resampled_rate * (t[1] - t[0]))  # in Hz
    for num_aliases in range(1, 5):
        fig, ax = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
        for n in range(-num_aliases, num_aliases + 1):
            shifted_frequencies = frequencies - n * sampling_frequency
            ax.plot(shifted_frequencies, np.log(1 + np.abs(Y)), label=f'Alias n={n}', linewidth=1)
        ax.set_title('Aliased Spectra due to Sampling')
        ax.set_xlabel('Frequency [Hz]')
        ax.set_ylabel('Magnitude')
        # set plot limits
        ax.set_xlim(min(frequencies) - 1, max(frequencies) + 1)
        # ax.set_ylim(0, 6)
        ax.legend()
        plt.savefig(f'figs/samp_and_rec/aliased_spectra_{resampled_rate}_{num_aliases}.png', dpi=300)
        plt.show()

        if resampled_rate == 2 and num_aliases == 1:
            fig, ax = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
            shifted_frequencies = frequencies
            l4 = len(frequencies)//4
            ax.plot(shifted_frequencies[-l4:], np.log(1 + np.abs(Y))[l4:-2*l4-1], label=f'Alias n=-1', linewidth=1)
            ax.plot(shifted_frequencies[l4:-l4-1], np.log(1 + np.abs(Y))[l4:-l4-1], label=f'Alias n=0', linewidth=1)
            ax.plot(shifted_frequencies[:l4], np.log(1 + np.abs(Y))[2*l4:-l4-1], label=f'Alias n=1', linewidth=1)
            # veritcal line at l4 and -l4
            ax.vlines([shifted_frequencies[l4], shifted_frequencies[-l4-1]], ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='gray', linestyle='--', linewidth=0.8)
            ax.set_title('Aliased Spectra due to Sampling(No Overlap)')
            ax.set_xlabel('Frequency [Hz]')
            ax.set_ylabel('Magnitude')
            # set plot limits
            ax.set_xlim(min(frequencies) - 1, max(frequencies) + 1)
            # ax.set_ylim(0, 6)
            ax.legend()
            plt.savefig(f'figs/samp_and_rec/aliased_spectra_{resampled_rate}_{num_aliases}_nooverlap.png', dpi=300)
            plt.show()

            Y_nooverlap = np.zeros_like(Y)
            Y_nooverlap[-l4:] = Y[l4:-2*l4-1]
            Y_nooverlap[l4:-l4-1] = Y[l4:-l4-1]
            Y_nooverlap[:l4] = Y[2*l4:-l4-1]
            fig, ax = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
            ax.plot(frequencies, np.log(1 + np.abs(Y_nooverlap)), color='red', linewidth=1)
            ax.set_title('Frequency Spectrum of Sampled Signal (No Overlap) ifft')
            ax.set_xlabel('Frequency [Hz]')
            ax.set_ylabel('Magnitude')
            # set plot limits
            ax.set_xlim(min(frequencies) - 1, max(frequencies) + 1)
            # ax.set_ylim(0, 6)
            ax.legend()
            plt.savefig(f'figs/samp_and_rec/sampled_signal_spectrum_{resampled_rate}_nooverlap_ifft_spectrum.png', dpi=300)
            plt.show()

            # inverse fft
            y_nooverlap:np.ndarray = fft.ifft(fft.fftshift(Y_nooverlap), n=len(frequencies)*256)
            n_t = np.linspace(0, t_max, len(y_nooverlap))
            y_nooverlap = np.abs(y_nooverlap)/(n_t[1]-n_t[0])  # scale by dt for CTFT units
            fig, ax = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
            ax.plot(t, y, label='Original Signal', color='lightgray', linewidth=1)
            ax.plot(n_t, y_nooverlap, label='Reconstructed Signal (No Overlap)', color='blue', linewidth=0.5)
            # ax.scatter(t_samples_pos, y_samples_pos, label='Sampled Signal', color='red', s=20)
            ax.set_xlim(0, t_max)
            # ax.set_ylim(-0.1, 1.2)
            ax.set_title(f'Reconstructed Signal (No Overlap) from IFFT')
            ax.set_xlabel('Time [s]')
            ax.set_ylabel('Amplitude')
            ax.legend()
            plt.savefig(f'figs/samp_and_rec/reconstructed_signal_{resampled_rate}_nooverlap_ifft.png', dpi=300)
            plt.show()


    # Reconstruct signal using box function
    def box_kernel(width):
        return np.ones(width)

    def hat_kernel(width):
        t_kernel = np.linspace(-1, 1, width, endpoint=False)
        return np.maximum(0, 1 - np.abs(t_kernel))

    def gaussian_kernel(width, resampled_rate, sigma):
        w = width // 2
        t_kernel = np.linspace(-w, w, width, endpoint=False)/resampled_rate
        g = np.exp(-0.5 * (t_kernel / sigma) ** 2)
        g = g / (sigma * np.sqrt(2 * np.pi))
        return g

    for kernel_name in ["box", "hat", "gaussian"]:
        kernel = []
        if kernel_name == "box":
            kernel = box_kernel(resampled_rate)
        elif kernel_name == "hat":
            kernel = hat_kernel(2*resampled_rate)
        elif kernel_name == "gaussian":
            kernel = gaussian_kernel(4*resampled_rate, resampled_rate, sigma=0.5)

        # plt.figure(figsize=(8, 4))
        # fig, ax = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
        # ax.plot(kernel, color='green', linewidth=1)
        # ax.set_xlabel('Samples')
        # ax.set_ylabel('Amplitude')
        # plt.savefig(f'figs/samp_and_rec/{kernel_name}_reconstruction_kernel_{resampled_rate}.png', dpi=300)
        # plt.show()
            
        y_reconstructed = np.convolve(y_samples, kernel, mode='same')
        # keep only the sampled values (non-zero)
        t_reconstructed_pos = t[y_samples != 0]
        y_reconstructed_pos = y_samples[y_samples != 0]
        fig, ax = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
        ax.plot(t, y, label='Original Signal', color='lightgray', linewidth=1)
        ax.plot(t, y_reconstructed, label='Reconstructed Signal', color='blue')
        ax.scatter(t_reconstructed_pos, y_reconstructed_pos, label='Sampled Signal', color='red', s=20)
        ax.set_xlim(0, t_max)
        ax.set_ylim(-0.1, 1.2)
        ax.set_title(f'Reconstructed Signal with {kernel_name} Kernel')
        ax.set_xlabel('Time [s]')
        ax.set_ylabel('Amplitude')
        ax.legend()
        plt.savefig(f'figs/samp_and_rec/reconstructed_signal_{kernel_name}_{resampled_rate}.png', dpi=300)
        plt.show()

        # fft reconstructed
        Y_reconstructed = fft.fftshift(fft.fft(y_reconstructed))
        fig, ax = plt.subplots(1, 1, figsize=(8, 4), constrained_layout=True)
        ax.plot(frequencies, np.log(1 +np.abs(Y)), color='lightgray', linewidth=1)
        ax.plot(frequencies, np.log(1+np.abs(Y_reconstructed)), color='red', linewidth=1)
        ax.set_xlim(min(frequencies) - 1, max(frequencies) + 1)
        ax.set_ylim(0, 6)
        ax.set_title(f'Frequency Spectrum of Reconstructed Signal with {kernel_name} Kernel')
        ax.set_xlabel('Frequency [Hz]')
        ax.set_ylabel('Magnitude')
        plt.savefig(f'figs/samp_and_rec/reconstructed_spectrum_{kernel_name}_{resampled_rate}.png', dpi=300)
        plt.show()

In [ ]:
plt.close('all')

def f2d(x, y):
    return 0.5 + 0.5*np.sin(2 * np.pi * x * x * y * y)



for n in [2048, 1024, 128]:
    # sample f
    x = np.linspace(0, 4, n + 1, endpoint=False) + 0.5
    y = np.linspace(0, 4, n + 1, endpoint=False) + 0.5
    X, Y = np.meshgrid(x, y)
    Z = f2d(X, Y)

    # compute Z dft
    F_Z = fft.fftshift(fft.fft2(Z))# * (x[1]-x[0]) * (y[1]-y[0])  # scale by dx*dy for CTFT units
    F_frequencies_x = fft.fftshift(fft.fftfreq(len(x), d=(x[1]-x[0])))
    F_frequencies_y = fft.fftshift(fft.fftfreq(len(y), d=(y[1]-y[0])))
    # select frequency range
    mask_x = (F_frequencies_x >= -200) & (F_frequencies_x <= 200)
    mask_y = (F_frequencies_y >= -200) & (F_frequencies_y <= 200)
    F_frequencies_x = F_frequencies_x[mask_x]
    F_frequencies_y = F_frequencies_y[mask_y]
    F_Z = F_Z[np.ix_(mask_y, mask_x)]

    
    fig, ax = plt.subplots(1, 1, figsize=(6, 5), constrained_layout=True)
    contour = ax.contourf(X, Y, Z, levels=50, cmap='viridis')
    # contour = ax.contour(X, Y, Z, levels=np.linspace(0, 1, 5), cmap='viridis')
    fig.colorbar(contour, ax=ax, label='f(x, y)')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('2D Function Sampling')
    plt.savefig(f'figs/samp_and_rec/function_f_2D_{n}.png', dpi=300)
    plt.show()

    # plot Z as image
    fig, ax = plt.subplots(1, 1, figsize=(6, 5), constrained_layout=True)
    im = ax.imshow(Z, extent=(0, 4, 0, 4), origin='lower', cmap='gray', aspect='auto')
    fig.colorbar(im, ax=ax, label='f(x, y)')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('2D Function Sampling (Image)')
    plt.savefig(f'figs/samp_and_rec/function_f_2D_image_{n}.png', dpi=300)
    plt.show()

    # plot FT magnitude as image
    fig, ax = plt.subplots(1, 1, figsize=(6, 5), constrained_layout=True)
    im = ax.imshow(np.log(1 + np.abs(F_Z)), extent=(F_frequencies_x[0], F_frequencies_x[-1], F_frequencies_y[0], F_frequencies_y[-1]), 
                origin='lower', cmap='gray', aspect='auto')
    fig.colorbar(im, ax=ax, label='Magnitude')
    ax.set_xlabel('Frequency x [Hz]')
    ax.set_ylabel('Frequency y [Hz]')
    ax.set_title('2D Function Frequency Spectrum (Image)')
    plt.savefig(f'figs/samp_and_rec/function_f_2D_spectrum_image_{n}.png', dpi=300)
    plt.show() 

In [ ]:
plt.close('all')
def f2d_conv_k(x, y, dx, dy, kernel, kernel_width, num_samples=32):
    """ Use month-carlo integration to convolve f with kernel"""

    # generate random samples in the kernel support
    rand_x = np.random.uniform(-0.5, 0.5, num_samples)
    rand_y = np.random.uniform(-0.5, 0.5, num_samples)
    result = 0
    for kx, ky in zip(rand_x, rand_y):
        k_value = kernel(kx*kernel_width)*kernel(ky*kernel_width)
        f_value = f2d(x - kx*dx*kernel_width, y - ky*dy*kernel_width)
        result += f_value * k_value
    return (kernel_width**2 * result) / num_samples

n = 128
for kernel_name in ["box", "hat", "gaussian", "gaussian2"]:
    kernel_width = 2
    kernel_func = lambda t: t
    if kernel_name == "box":
        kernel_func = lambda t: box(t, -0.5, 0.5)
        kernel_width = 1
    elif kernel_name == "hat":
        kernel_func = lambda t: hat(t, 0, 1)
        kernel_width = 2
    elif kernel_name == "gaussian":
        kernel_func = lambda t: gaussian(t, 0, 0.5, -1.5, 1.5)
        kernel_width = 3
    elif kernel_name == "gaussian2":
        kernel_func = lambda t: gaussian(t, 0, 1.5, -5, 5)
        kernel_width = 6

    # sample f_conv_k
    x = np.linspace(0, 4, n, endpoint=False) + 0.5
    y = np.linspace(0, 4, n, endpoint=False) + 0.5
    X, Y = np.meshgrid(x, y)

    for num_samples in [16, 64, 256]:
        Z = f2d_conv_k(X, Y, (x[1]-x[0]), (y[1]-y[0]), kernel_func, kernel_width=kernel_width, num_samples=num_samples)

        # plot Z as image
        fig, ax = plt.subplots(1, 1, figsize=(6, 5), constrained_layout=True)
        im = ax.imshow(Z, extent=(0, 4, 0, 4), origin='lower', cmap='gray', aspect='auto', vmin=0, vmax=1)
        fig.colorbar(im, ax=ax, label='f_conv_k(x, y)')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_title(f'2D Function Convolved with {kernel_name.capitalize()} Kernel with {num_samples} Samples')
        plt.savefig(f'figs/samp_and_rec/function_f_2D_convolved_{kernel_name}_image_{n}_samples_{num_samples}.png', dpi=300)
        plt.show()

        # plot Z spectrum as image
        F_Z = fft.fftshift(fft.fft2(Z))  # * (x[1]-x[0]) * (y[1]-y[0])  # scale by dx*dy for CTFT units
        F_frequencies_x = fft.fftshift(fft.fftfreq(len(x), d=(x[1]-x[0])))
        F_frequencies_y = fft.fftshift(fft.fftfreq(len(y), d=(y[1]-y[0])))
        fig, ax = plt.subplots(1, 1, figsize=(6, 5), constrained_layout=True)
        im = ax.imshow(np.log(1 + np.abs(F_Z)), extent=(F_frequencies_x[0], F_frequencies_x[-1], F_frequencies_y[0], F_frequencies_y[-1]), 
                    origin='lower', cmap='gray', aspect='auto')
        fig.colorbar(im, ax=ax, label='Magnitude')
        ax.set_xlabel('Frequency x [Hz]')
        ax.set_ylabel('Frequency y [Hz]')
        ax.set_title(f'2D Function Convolved with {kernel_name.capitalize()} Kernel Frequency Spectrum with {num_samples} Samples')
        plt.savefig(f'figs/samp_and_rec/function_f_2D_convolved_{kernel_name}_spectrum_image_{n}_samples_{num_samples}.png', dpi=300)
        plt.show()